# MLiP MDS am Beispiel PSF
Kurs Maschinelles Lernen in der Produktion

#### In diesem Notebook wird das MDS anhand des Anwendungsbeispiels Profilschienenführung (PSF) geübt. 

Linearführungen und Profilschienenführungen sind für einen nicht unbedeutenden Teil der Ausfälle bei Werkzeugmaschinen.
Typische Fehlerfälle die dabei auftreten sind Mangelschmierungen, Pittings an Laufbahnen oder an den Wälzkörpern. In diesem Beispiel ist aber der grundlegende Zustand der PSF von Interesse, d.h. ist der Zustand fehlerfrei (OK) oder machen sich Verschleißerscheinungen (Ausfall) deutlich. 

Für dieses Beispiel wurden mehrere Profilschienen unter Belastung bis zu ihrem Lebensende verfahren. Die dabei auftretenden Fehler sind natürlich entstanden. Dementsprechend unterscheiden Sie sich sowohl in der Art als auch in der Stärke. 

Während der Versuche wurde mittels einem 3-achsigen MEMS-Sensor die Beschleunigungen, in Verfahrrichtung (Acc_X)
in Richtung oder entgegen Erdmittelpunkt (Acc_Y) und orthogonal zu beiden in Richtung des Seitennormalenvektor des Versuchsstands (Acc_Z) aufgenommen. Die entsprechenden Zeitreihen:
<table><tr>
<td> <img src="MLiP_PSF_Messfahrt_gut.png" alt="Status OK" style="width: 350px;"/> </td>
<td> <img src="MLiP_PSF_Messfahrt_ausfall.png" alt="Status Ausfall" style="width: 350px;"/> </td>
</tr></table>
Für diese Aufgabe wurde nur die Beschleunigung in Verfahrrichtung (Acc_X) verwendet. Deren Werte wurden bereits zu Features zusammengefasst. Ziel ist es nun diese Features mit Hilfe des MDS auf so wenige Dimensionen wie möglich zu reduzieren. 


### Data-Mining-Prozess:

![Bild konnte nicht geladen werden! 1. Daten erfassen - 2. Daten erkunden - 3. Daten vorbereiten - 4. Modelle bilden - 5. Modelle validieren - 6. Modell testen](Prozess_Modellentwicklung_v2.png "Title")

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

(Wähle für alle Notebooks in allen Übungen immer die gleiche Zahl (z.B. den Tag deines Geburtstags), dann sind die Ergebnisse der verschiedenen Machine-Learning-Verfahren vergleichbar da dann alle Notebooks mit der "gleichen" Folge an Zufallszahlen arbeiten)

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 1. Daten erfassen - Daten importieren

In [ ]:
# Lade Datensatz
df = pd.read_csv("CM_PSF_Features.csv")

### 2. Daten erkunden

In [ ]:
# Datensatz anzeigen
df.head(10)

In [ ]:
df.describe()

### 3.1 Daten vorbereiten - Auswahl der Daten
Es werden 6 Prüflinge aus dem Datensatz ausgewählt. Die MDS für den Gesamtdatensatz dauert die Berechnung zu lange. 

In [ ]:
# Aufteilen der Daten in X y
selected_df = df[df.Pruefling_ID.isin([107, 108, 109, 126, 127, 128])]
selected_df = selected_df.assign(uID = df.Pruefling_ID *10000 + df.Messfahrt_ID ).reset_index(drop=True)
X_train = selected_df.drop(columns=['Status', 'Messfahrt_ID', 'Pruefling_ID'])

### 3.2 Daten vorbereiten - Daten normieren
 Für dieses Beispiel wurde der MinMaxScaler vorbereitet.

Es können auch andere Scaler verwendet werden, die müssen entsprechend importiert werden und der Befehl scaler = ... muss angepasst werden. 

In [ ]:
# Min-Max-Skalierung der Daten
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train.values), index=X_train.index, columns=X_train.columns)

### 4.1 Modell bilden - Modell Importieren

In [ ]:
# Importieren des Modells
from sklearn.manifold import MDS

Beschreibung der Hyperparameter:
https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html

### 4.1 Modelle bilden - Distanzmetriken berechnen
Dieser Schritt ist eigentlich nur notwendig, wenn nicht die euklidische Metrik als Abstandsmaß gewünscht ist. 

__[OPTIONAL]__ Nur nötig wenn für 'dissimilarity' nicht 'euclidean' gewählt wird.

In [ ]:
# [OPTIONAL] Nur nötig wenn für 'dissimilarity' nicht 'euclidean' gewählt wird.

# Erstelle eigenes Maß für Verschiedenheit
from sklearn.metrics import pairwise as metr

euclidean_similarities = metr.euclidean_distances(X_train)
cosine_similarities = metr.cosine_distances(X_train)
manhatten_similarities = metr.manhattan_distances(X_train)

Ausage der Distanzmetrik für die Abstände bei der euklidischen Metrik

In [ ]:
# Darstellung der euklidischen Abstände als DataFrame
pd.DataFrame(euclidean_similarities, index=selected_df.uID, columns=selected_df.uID)

### 4.2 Modelle bilden - Modell trainieren
In diesem Schritt erfolgt die Durchführung der MDS. 

TODO:
* Bestimme die Anzahl der Dimensionen, n_components, auf die skaliert werden soll. 

Notiz:
* Falls 'dissimilarity' = 'euclidean': fit_transform mit X_train
* Falls 'dissimilarity' = 'precomputed': fit_transform mit [euclidean_similarities, manhatten_similarities, cosine_similarities] 

Ausgabe: 
* Dauer der Berechnung

In [ ]:
# Modell erstellen
model = MDS(dissimilarity="euclidean",
            n_components=TODO,
            random_state=my_seed,
            n_jobs=-2)

# Variante für andere Metriken
#model = MDS(dissimilarity="precomputed",
#            n_components=TODO,
#            random_state=my_seed,
#            n_jobs=-2)

# Scalierung durchführen
start_timer = time.monotonic()
X_train_trans = model.fit_transform(X_train)

# Variante für andere Metriken, hier z.B. für manhatten_similarities
# X_train_trans = model.fit_transform(manhatten_similarities)

print("\nDauer " + str("%.1f" % (time.monotonic() - start_timer))
      + " Sekunden")

### 5.1 Modelle validieren - Visuelle Bewertung

Um die Qualität/Güte des Modells zu bestimmen, wird diese visuell überprüft.

<font color='red'>Die Visualisierungen funktionieren nur bei Modellen mit n_components = 2 oder 3!</font>

In [ ]:
# Visuelle Darstellung für Transformation auf 2 bzw. 3 Komponenten
X_transformed = np.transpose(X_train_trans)
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
mycolors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
y_colors = [mycolors[status] for status in encoder.fit_transform(selected_df["Status"])]
if len(X_transformed) == 2:
    fig = plt.figure(figsize=(16, 10))
    plt.scatter(X_transformed[0], X_transformed[1],
                s=200,        
                alpha=0.5,
                c=y_colors,
                edgecolors="black",
                marker="o")
    plt.title("Visualisierung: Multidimensional-Scaling auf 2 Input-Parameter")
    plt.show()
    
elif len(X_transformed)== 3:
    # 3D-Plot Ergebnis Clustering
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib.patches import Patch
    
    # 3D Plot-Befehl
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection="3d")
    legend = []

    # Plot der Datenpunkte mit Einfärbung
    ax.scatter(X_transformed[0], X_transformed[1], X_transformed[2],
               c=y_colors,
               marker="o",
               s=140)
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_zlabel("x3")
    plt.show()
    
else:
    print(str(len(X_transformed))
        + " statt 2 oder 3 Komponenten, für eine grafische Darstellung ein Modell mit n_components = 2 oder 3 trainieren.")
    

Komplexere Visualisierung mit unterschiedlichen Markern je Prüfling

In [ ]:
# Visuelle Darstellung für Transformation auf 2 bzw. 3 Komponenten
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
encoder.fit(selected_df["Status"])
mycolors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
markers = ['.', 'o', 'v', 's', 'P', '*']
if len(X_transformed) == 2:
    fig = plt.figure(figsize=(16, 10))
    for idx, Pruefling_ID in enumerate(selected_df.Pruefling_ID.unique()):
        plt.scatter(X_train_trans[selected_df.Pruefling_ID==Pruefling_ID, 0], X_train_trans[selected_df.Pruefling_ID==Pruefling_ID,1],
                    s=200,        
                    alpha=0.5,
                    c=[mycolors[status] for status in encoder.transform(selected_df.loc[selected_df.Pruefling_ID==Pruefling_ID,"Status"])],
                    edgecolors="black",
                    marker=markers[idx],
                    label=Pruefling_ID)
    plt.title("Visualisierung: Multidimensional-Scaling auf 2 Input-Parameter")
    plt.legend()
    plt.show()
    
elif len(X_transformed)== 3:
    # 3D-Plot Ergebnis Clustering
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib.patches import Patch
    
    # 3D Plot-Befehl
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection="3d")
    
    for idx, Pruefling_ID in enumerate(selected_df.Pruefling_ID.unique()):

    # Plot der Datenpunkte mit Einfärbung
        ax.scatter(X_train_trans[selected_df.Pruefling_ID==Pruefling_ID, 0], 
                   X_train_trans[selected_df.Pruefling_ID==Pruefling_ID, 1],
                   X_train_trans[selected_df.Pruefling_ID==Pruefling_ID, 2],
                   c=[mycolors[status] for status in encoder.transform(selected_df.loc[selected_df.Pruefling_ID==Pruefling_ID,"Status"])],
                   marker=markers[idx],
                   s=140,                     
                   edgecolors="black",
                   label=Pruefling_ID)
    
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_zlabel("x3")
    plt.legend()
    plt.show()
    
else:
    print(str(len(X_transformed))
        + " statt 2 oder 3 Komponenten, für eine grafische Darstellung ein Modell mit n_components = 2 oder 3 trainieren.")
    

### 5.2 Modelle validieren - Kennzahlen berechnen

In [ ]:
# Berechne Stress-Wert
from sklearn import metrics

similarities = metrics.euclidean_distances(X_train)
dist_lowtriang = np.tril(similarities, -1)
dist_quadr = np.square(dist_lowtriang)
print("Stress-Wert:\t" + str("%.3f" % np.sqrt(model.stress_ / np.sum(dist_quadr))))

### 7. Visualisierung
In diesem Schritt werden automatisch mehrere MDS mit unterschiedlicher Anzahl von Dimensionen, n_components, berechnet. Für jede wird der Stress Wert bestimmt und anschließend als Grafik ausgegeben. 

__Achtung:__ Die Berechnung kann etwas dauern.

In [ ]:
# Berechne Stress 1-Wert für Modelle
n_dims = [2, 3, 4, 5, 6]
results = []
for n_dim in n_dims:
    # Modell erstellen
    model = MDS(dissimilarity="euclidean",
                n_components=n_dim,
                random_state=my_seed, 
                n_jobs=-2)
    # Skalierung durchführen
    X_train_trans = model.fit_transform(X_train)
    similarities = metrics.euclidean_distances(X_train)
    dist_lowtriang = np.tril(similarities, -1)
    dist_quadr = np.square(dist_lowtriang)
    stress_1 = np.sqrt(model.stress_ / np.sum(dist_quadr))
    results.append(stress_1)

# Visualisierung
plt.figure(figsize=(16, 10))
plt.plot(n_dims, results, "o--")
plt.plot([2, 6], [0.2, 0.2])
plt.plot([2, 6], [0.1, 0.1])
plt.plot([2, 6], [0.05, 0.05])
plt.plot([2, 6], [0.025, 0.025])
plt.xlabel("Anzahl Dimensionen nach Transformation")
plt.ylabel("Stress-Wert")
plt.title("Stress 1-Wert über Anzahl an Dimensionen")
plt.legend(["data", "gering", "ausreichend", "gut", "ausgezeichnet"], loc='upper right')
plt.show()